### Perform windowed-CRQA on actual pairs

In [1]:
# Import necessary libraries
import pandas as pd
from utils_rqa.crossRQA import crossRQA
import os 
import numpy as np
from tqdm.notebook import tqdm

In [2]:
# Features (time series variables) to include in the CRQA
pos_cols = ['head_ed_pos',
            'headRel_ed_pos',
            'body_ed_pos'
            ]

acc_cols = ['head_ed_acc', 'body_ed_acc',
            'headRel_ed_acc']

vel_cols = ['head_ed_vel', 'headRel_ed_vel', 'body_ed_vel', 
            'whole_all_vel', 'head_all_vel', 'body_all_vel',
            'head_movement_norm', 'body_movement_norm', 'full_body_movement_norm']

analysed_cols = ['headRel_ed_vel', 'body_ed_vel']

In [ ]:
roles = pd.read_csv('partner_role_metadata.csv') # Load partner role metadata

# Set directories
directory = "/..../Processed_Timeseries" # Set directory to processed timeseries here.
output_dir = "crqa_output" # Set output directory here
os.makedirs(output_dir, exist_ok=True)

# Select only "P1" files
files = [f for f in os.listdir(directory) if f.split('_')[1] == 'P1']

# Choose which set of columns to use:
cols = analysed_cols 

In [ ]:
"""
Perform Windowed-CRQA. If a window is missing for one of the partners, window is skipped. 
Designed to loop through multiple parameter settings. 
Returns a csv file (CrossRqa_win_delay{d}_dim{m}_rad{r}_minl{crqa_params['minl']}.csv) for each parameter setting combination

Parameters used for the paper include:
    - Delay = 5
    - Embedding = 4
    - Radii = 0.25
    - min line = 2
"""

# Parameters for delay embedding + recurrence analysis
delays = [5] 
embedding = [4]
radii = [0.25] 

# Loop through parameters and files to perform windowed cross-recurrence quantification analysis (CRQA)
for d in tqdm(delays, desc='Delays Processed'):
    for m in tqdm(embedding, desc='Embedding Processed', leave=False):
        for r in tqdm(radii, desc="Radii Processed", leave=False):
            results = [] # Store results for all couples/trials/windows

            # RQA parameters
            crqa_params = {
                'norm': 2, # 2 = zscore, 1 = unit
                'eDim': m,
                'tLag': d,
                'rescaleNorm': 1, # 1 = mean rescale, 2 = max
                'radius': r,
                'tw': 0,
                'minl': 2,
                'doPlots': False,
                #'plotMode': 'rp-timeseries', # 'rp' or 'rp-timeseries'
                'pointSize': 2, 
                'saveFig': False,
                'showMetrics': False,
                'doStatsFile': False
            }
            # Loop through files
            for file in files: 
                print(file)
                couple = file.split('_')[0] # Extract couple ID from filename
                trial = file.split('_')[4]  # Extract trial number from filename

                # Load participant 1 data
                p1_data_path = os.path.join(directory, file)
                p1_data = pd.read_csv(p1_data_path)

                 # Load corresponding participant 2 data
                p2_data_path = os.path.join(directory, file.replace('P1', 'P2'))

                # Check if the P2 file exists before attempting to read it
                if os.path.exists(p2_data_path):
                    p2_data = pd.read_csv(p2_data_path)
                else:
                    print(f"File does not exist: {file.replace('P1', 'P2')}")

                # Debug print of paired files
                print(f"selected file: {file}")
                print(f"partner file: {file.replace('P1', 'P2')}")

                if trial != 'trial0':
                    role = roles[(roles['couple'] == int(couple)) & (roles['trial'] == trial)]
                    p1_role = role['p1_role'].values[0]
                    p2_role = role['p2_role'].values[0]

                    if p1_role == 0 and p2_role == 1:
                        agent = p1_data
                        target = p2_data
                    elif p1_role == 1 and p2_role == 0:
                        agent = p2_data
                        target = p1_data
                    else:
                        print(f"Roles not properly defined for couple {couple}, trial {trial}")
                        continue
                elif trial == 'trial0':
                    agent = p1_data
                    target = p2_data

                # Extract window indices for both participants 
                p1_win_index = p1_data['Window_Index'].values
                p2_win_index = p2_data['Window_Index'].values

                # Find common window indices between both participants
                matching_windows = np.intersect1d(p1_win_index, p2_win_index)

                # Subset data to only include matching windows
                p1_col = p1_data[p1_data['Window_Index'].isin(matching_windows)]
                p2_col = p2_data[p2_data['Window_Index'].isin(matching_windows)]

                # Skip if window counts do not match
                if len(p1_col) != len(p2_col):
                    print(f"Unequal length. Couple = {couple}, trial = {trial}")
                    continue
    
                # Loop through each matching window and perform CRQA
                i = 0
                for idx in matching_windows:
                    i += 1
                    p1_window = p1_col[p1_col['Window_Index'] == idx][cols].iloc[1:]
                    p2_window = p2_col[p2_col['Window_Index'] == idx][cols].iloc[1:]

                    file_info = {
                        'couple': couple,
                        'trial': trial,
                        'window_index': idx,
                        }
                    
                    try: 
                        output = crossRQA(p1_window, p2_window, crqa_params, file_info=file_info)
                    except Exception as ex:
                        print('error')
                        file_info['err_code'] = 1
                        output = file_info
                        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                        message = template.format(type(ex).__name__, ex.args)
                        print(f'{message}: For file: {file}')
                    
                    results.append(output)

            # Save results for current parameter combination
            csv_df = pd.DataFrame(results)
            csv_df.to_csv(f"{output_dir}/CrossRqa_win_delay{d}_dim{m}_rad{r}_minl{crqa_params['minl']}.csv", index=False)      

            print('Windowed CRQA analysis and plotting completed successfully!')